## Compare the assemblies in GenBank, GTDB, and RAST

Each has a different set. What are the unions and intersections?

In [1]:
# A lot of this is not used, but we import it so we have it later!
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import pandas as pd
import seaborn as sns
import numpy as np

import math
import re

from PhiSpyAnalysis import theils_u, DateConverter, printmd
from PhiSpyAnalysis import read_phages, read_gtdb, read_checkv, read_base_pp, read_categories, read_metadata, read_gbk_metadata

from scipy.stats import pearsonr, f_oneway
from sklearn.linear_model import LinearRegression
from sklearn import decomposition
from sklearn.ensemble import RandomForestClassifier

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd, tukeyhsd, MultiComparison
from statsmodels.multivariate.manova import MANOVA


import subprocess
import gzip


In [2]:
rv = re.compile('^\\w+')
def remove_ver(x):
    return rv.search(str(x)).group()

# GTDB



In [3]:
# GTDB
gtdb = read_gtdb()
gtdb['assembly_nover'] = gtdb['assembly_accession'].apply(remove_ver)
gtdb[['assembly_accession', 'assembly_nover']]

,assembly_accession,assembly_nover
0,GCA_000006155.2,GCA_000006155
1,GCA_000007385.1,GCA_000007385
2,GCA_000008605.1,GCA_000008605
3,GCA_000010565.1,GCA_000010565
4,GCA_000013845.2,GCA_000013845
...,...,...
191522,GCA_902166935.1,GCA_902166935
191523,GCA_902166945.1,GCA_902166945
191524,GCA_902167295.1,GCA_902167295
191525,GCA_902167305.1,GCA_902167305


In [13]:
gtdb

,accession,ambiguous_bases,checkm_completeness,checkm_contamination,checkm_marker_count,checkm_marker_lineage,checkm_marker_set_count,checkm_strain_heterogeneity,coding_bases,coding_density,...,ssu_silva_blast_bitscore,ssu_silva_blast_evalue,ssu_silva_blast_perc_identity,ssu_silva_blast_subject_id,ssu_silva_taxonomy,total_gap_length,trna_aa_count,trna_count,trna_selenocysteine_count,assembly_nover
0,GB_GCA_000006155.2,1916,93.12,0.00,1171,g__Bacillus (UID902),324,0.0,4305660,80.178992,...,NaN,NaN,NaN,NaN,NaN,42300,16,31,0,GCA_000006155
1,GB_GCA_000007385.1,0,99.82,0.00,481,c__Gammaproteobacteria (UID4202),276,0.0,4190634,84.805944,...,2846.0,0.0,100.000,JXEG01000201.4293.5839,Bacteria;Proteobacteria;Gammaproteobacteria;Xa...,0,20,53,0,GCA_000007385
2,GB_GCA_000008605.1,67,100.00,0.00,235,f__Spirochaetaceae (UID2512),124,0.0,1048744,92.155875,...,2854.0,0.0,100.000,CP003679.231299.232859,Bacteria;Spirochaetes;Spirochaetia;Spirochaeta...,0,20,45,0,GCA_000008605
3,GB_GCA_000010565.1,0,100.00,0.63,295,p__Firmicutes (UID1022),158,0.0,2608397,86.217312,...,2819.0,0.0,100.000,AP009389.1049070.1050595,Bacteria;Firmicutes;Clostridia;Clostridiales;P...,0,20,51,1,GCA_000010565
4,GB_GCA_000013845.2,0,100.00,0.00,332,o__Clostridiales (UID1375),124,0.0,2428396,82.037966,...,2787.0,0.0,100.000,CP000312.233320.234832,Bacteria;Firmicutes;Clostridia;Clostridiales;C...,0,20,95,1,GCA_000013845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191522,RS_GCF_902166935.1,0,99.62,0.09,1312,g__Klebsiella (UID5140),336,0.0,4894244,87.293148,...,2837.0,0.0,100.000,MAPL01000082.143.1696,Bacteria;Proteobacteria;Gammaproteobacteria;En...,0,20,87,1,GCA_902166935
191523,RS_GCF_902166945.1,0,99.94,0.72,1162,f__Enterobacteriaceae (UID5121),336,0.0,5162721,86.540042,...,2837.0,0.0,100.000,CP016813.16076.17629,Bacteria;Proteobacteria;Gammaproteobacteria;En...,0,20,86,1,GCA_902166945
191524,RS_GCF_902167295.1,0,99.51,0.00,303,p__Bacteroidetes (UID2591),203,0.0,5737447,90.122088,...,2747.0,0.0,99.933,JX101438.1.1490,Bacteria;Bacteroidetes;Bacteroidia;Chitinophag...,0,20,58,0,GCA_902167295
191525,RS_GCF_902167305.1,0,99.51,0.00,303,p__Bacteroidetes (UID2591),203,0.0,5738096,90.147873,...,2747.0,0.0,99.933,JX101438.1.1490,Bacteria;Bacteroidetes;Bacteroidia;Chitinophag...,0,20,58,0,GCA_902167305


In [5]:
# RAST
# the full data set. Don't try this at home!
# metadf = pd.read_csv("../small_data/patric_genome_metadata.tsv.gz", compression='gzip', header=0, delimiter="\t")
rast = read_metadata()
rast['assembly_nover'] = rast['assembly_accession'].apply(remove_ver)
rast[['assembly_accession', 'assembly_nover']]

,assembly_accession,assembly_nover
0,GCA_000003135.1,GCA_000003135
1,GCA_000003215.1,GCA_000003215
2,GCA_000003645.1,GCA_000003645
3,GCA_000003925.1,GCA_000003925
4,GCA_000003955.1,GCA_000003955
...,...,...
320171,GCF_900167595.1,GCF_900167595
320172,GCF_900167605.1,GCF_900167605
320173,GCF_900167615.1,GCF_900167615
320174,GCF_900167625.1,GCF_900167625


In [14]:
rast

,assembly_accession,genome_id,genome_name,organism_name,taxon_id,genome_status,strain,serovar,biovar,pathovar,...,temperature_range,optimal_temperature,salinity,oxygen_requirement,habitat,disease,comments,additional_metadata,isolation_date,assembly_nover
0,GCA_000003135.1,5.484803e+05,Bifidobacterium longum subsp. longum ATCC 55813,Bifidobacterium longum subsp. longum ATCC 55813,548480,WGS,ATCC 55813,NaN,NaN,NaN,...,Mesophilic,-,NaN,Anaerobic,Host-associated,NaN,Bifidobacterium longum subsp. longum ATCC 5581...,NaN,NaN,GCA_000003135
1,GCA_000003215.1,4.556313e+05,Clostridium difficile QCD-66c26,Clostridium difficile QCD-66c26,455631,WGS,QCD-66c26,NaN,NaN,NaN,...,Mesophilic,37,NaN,Anaerobic,Multiple,Antibiotic-associated Diarrhea;Pseudomembraneo...,Clostridium difficile QCD-66c26. was isolated ...,NaN,NaN,GCA_000003215
2,GCA_000003645.1,5.269733e+05,Bacillus cereus m1293,Bacillus cereus m1293,526973,WGS,m1293,NaN,NaN,NaN,...,Mesophilic,-,NaN,Aerobic,Multiple,Food poisoning,"Bacillus cereus m1293. This strain, also know...",NaN,NaN,GCA_000003645
3,GCA_000003925.1,5.269973e+05,Bacillus mycoides DSM 2048,Bacillus mycoides DSM 2048,526997,WGS,DSM 2048,NaN,NaN,NaN,...,NaN,-,NaN,NaN,NaN,NaN,Bacillus mycoides DSM 2048. Bacillus mycoides...,NaN,NaN,GCA_000003925
4,GCA_000003955.1,5.269943e+05,Bacillus cereus AH1273,Bacillus cereus AH1273,526994,WGS,AH1273,NaN,NaN,NaN,...,Mesophilic,-,NaN,Aerobic,Multiple,Food poisoning,Bacillus cereus AH1273. This strain was isola...,NaN,NaN,GCA_000003955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320171,GCF_900167595.1,1.496160e+03,Clostridioides difficile strain VRECD0186,NaN,1496,WGS,VRECD0186,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,These data are part of a pre-publication relea...,NaN,2015.294319,GCF_900167595
320172,GCF_900167605.1,1.147123e+06,Caloramator quimbayensis strain USBA 833,NaN,1147123,WGS,USBA 833,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Caloramator quimbayensis USBA 833,NaN,NaN,GCF_900167605
320173,GCF_900167615.1,1.496159e+03,Clostridioides difficile strain CD19,NaN,1496,WGS,CD19,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.sanger.ac.uk/resources/downloads/ba...,NaN,2014.295003,GCF_900167615
320174,GCF_900167625.1,1.496159e+03,Clostridioides difficile strain VRECD0076,NaN,1496,WGS,VRECD0076,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,These data are part of a pre-publication relea...,NaN,2014.295003,GCF_900167625


# GenBank

This assembly summary comes from [GenBank ftp site](ftp://ftp.ncbi.nlm.nih.gov/genomes/genbank/) and you want the [assembly_summary.txt](ftp://ftp.ncbi.nlm.nih.gov/genomes/genbank/bacteria/assembly_summary.txt) file explicitly from bacteria (but don't try and open the bacteria list in your browser!)

In [6]:
# GenBank
gbk = read_gbk_metadata()
gbk['assembly_nover'] = gbk['assembly_accession'].apply(remove_ver)
gbk[['assembly_accession', 'assembly_nover']]

c:\workspace\edwa0468\githubs\phispyanalysis\venv\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,assembly_accession,assembly_nover
0,GCA_900128725.1,GCA_900128725
1,GCA_008244535.1,GCA_008244535
2,GCA_011046815.1,GCA_011046815
3,GCA_011054035.1,GCA_011054035
4,GCA_011331185.1,GCA_011331185
...,...,...
949928,GCA_017746755.1,GCA_017746755
949929,GCA_017746715.1,GCA_017746715
949930,GCA_013266695.1,GCA_013266695
949931,GCA_015831295.1,GCA_015831295


In [15]:
gbk

,assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,...,genome_rep,seq_rel_date,asm_name,submitter,gbrs_paired_asm,paired_asm_comp,ftp_path,excluded_from_refseq,relation_to_type_material,assembly_nover
0,GCA_900128725.1,PRJEB15400,SAMEA4556317,NaN,na,9.0,9.0,Buchnera aphidicola,strain=BCifornacula,2912,...,Full,2016/11/25,BCifornacula_v1.0,Centre de Biologie pour la Gestion des Populat...,GCF_900128725.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/900...,NaN,NaN,GCA_900128725
1,GCA_008244535.1,PRJNA562146,SAMN12634886,VTFL00000000.1,na,14.0,14.0,Dictyoglomus thermophilum,strain=PYS_80_B,NaN,...,Full,2019/09/02,ASM824453v1,Agharkar Research Institute,GCF_008244535.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/008...,NaN,NaN,GCA_008244535
2,GCA_011046815.1,PRJNA480137,SAMN09639436,DSOW00000000.1,na,14.0,14.0,Dictyoglomus thermophilum,NaN,SpSt-35,...,Full,2020/03/02,ASM1104681v1,The University of Hong Kong,na,na,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/011...,derived from metagenome,NaN,GCA_011046815
3,GCA_011054035.1,PRJNA480137,SAMN09639292,DSJI00000000.1,na,14.0,14.0,Dictyoglomus thermophilum,NaN,SpSt-22,...,Full,2020/03/02,ASM1105403v1,The University of Hong Kong,na,na,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/011...,derived from metagenome,NaN,GCA_011054035
4,GCA_011331185.1,PRJNA480137,SAMN09639525,DSSH00000000.1,na,14.0,14.0,Dictyoglomus thermophilum,NaN,SpSt-43,...,Full,2020/03/16,ASM1133118v1,The University of Hong Kong,na,na,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/011...,derived from metagenome,NaN,GCA_011331185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
949928,GCA_017746755.1,PRJNA647383,SAMN18473588,JAGHZP000000000.1,na,2821504.0,2821504.0,Cyanobacteria bacterium SID2,NaN,SID2,...,Full,2021/04/04,ASM1774675v1,University of Florida,na,na,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/017...,derived from metagenome; genus undefined,NaN,GCA_017746755
949929,GCA_017746715.1,PRJNA647383,SAMN18473589,JAGHZQ000000000.1,na,2821505.0,2821505.0,Cyanobacteria bacterium SBLK,NaN,SBLK,...,Full,2021/04/04,ASM1774671v1,University of Florida,na,na,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/017...,derived from metagenome; genus undefined,NaN,GCA_017746715
949930,GCA_013266695.1,PRJNA633965,SAMN14978626,JABRXF000000000.1,na,2821844.0,2821844.0,Hongsoonwoonella zoysiae,strain=SY4-7,NaN,...,Full,2020/06/03,ASM1326669v1,"BioPS Co., Ltd.",GCF_013266695.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/013...,NaN,NaN,GCA_013266695
949931,GCA_015831295.1,PRJNA679689,SAMN16843871,JADXDQ000000000.1,na,2822799.0,2822799.0,Streptococcus sp. NLN64,strain=NLN64,NaN,...,Full,2020/12/07,ASM1583129v1,Hainan Medical University-University,GCF_015831295.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/015...,NaN,NaN,GCA_015831295


In [7]:
phagesdf = read_phages(maxcontigs=-1) # this disables contig length filtering
phagesdf['assembly_nover'] = phagesdf['assembly_accession'].apply(remove_ver)
phagesdf

Please note that this was run with git commit 86420e1 that has 567,404 genomes parsed.
Initially there were 3,265,453 kept phages,but after filtering we kept 3,265,453 prophages from 567,399 genomes

,assembly_accession,assembly_name,Genome length,Contigs,Phage Contigs,Total Predicted Prophages,Kept,No phage genes,Not enough genes,bp prophage,assembly_nover
0,GCA_000043285.1,ASM4328v1,705557.0,1.0,1.0,2.0,0.0,2.0,0.0,0.0,GCA_000043285
1,GCA_000046685.1,ASM4668v1,1581384.0,1.0,1.0,18.0,3.0,0.0,15.0,97011.0,GCA_000046685
2,GCA_000046705.1,ASM4670v1,1931047.0,1.0,1.0,36.0,7.0,0.0,29.0,278490.0,GCA_000046705
3,GCA_000046845.1,ASM4684v1,3598621.0,1.0,1.0,13.0,3.0,4.0,6.0,83112.0,GCA_000046845
4,GCA_000047365.1,ASM4736v1,4840898.0,3.0,3.0,22.0,3.0,3.0,16.0,213427.0,GCA_000047365
...,...,...,...,...,...,...,...,...,...,...,...
567399,GCA_905187425.1,Xanthomonas_sp._CPBF_424_-_hybrid_assembly,4900930.0,1.0,1.0,46.0,7.0,9.0,30.0,214278.0,GCA_905187425
567400,GCA_905188235.1,ASM90518823v1,4963609.0,1.0,1.0,29.0,4.0,4.0,21.0,178993.0,GCA_905188235
567401,GCA_905219375.1,QI0054,3034314.0,1.0,1.0,12.0,4.0,3.0,5.0,178527.0,GCA_905219375
567402,GCA_905219385.1,QI0055,3034113.0,1.0,1.0,12.0,4.0,3.0,5.0,178526.0,GCA_905219385


In [12]:
phagesdf[phagesdf['Kept'] == 0]

,assembly_accession,assembly_name,Genome length,Contigs,Phage Contigs,Total Predicted Prophages,Kept,No phage genes,Not enough genes,bp prophage,assembly_nover
0,GCA_000043285.1,ASM4328v1,705557.0,1.0,1.0,2.0,0.0,2.0,0.0,0.0,GCA_000043285
5,GCA_000048645.1,ASM4864v1,3635495.0,2.0,2.0,4.0,0.0,4.0,0.0,0.0,GCA_000048645
6,GCA_000048665.1,ASM4866v1,3405519.0,2.0,2.0,5.0,0.0,5.0,0.0,0.0,GCA_000048665
19,GCA_000020245.1,ASM2024v1,1608548.0,1.0,1.0,11.0,0.0,6.0,5.0,0.0,GCA_000020245
22,GCA_000020305.1,ASM2030v1,1202435.0,1.0,1.0,4.0,0.0,0.0,4.0,0.0,GCA_000020305
...,...,...,...,...,...,...,...,...,...,...,...
567245,GCA_904067595.1,ASM90406759v1,6794181.0,68.0,55.0,46.0,0.0,21.0,25.0,0.0,GCA_904067595
567392,GCA_905143135.1,15-58d44,1158207.0,5.0,3.0,1.0,0.0,1.0,0.0,0.0,GCA_905143135
567394,GCA_905176665.1,D4_OalgB6SA,5465468.0,286.0,71.0,10.0,0.0,10.0,0.0,0.0,GCA_905176665
567395,GCA_905176675.1,G3_OalgB6SA,4272394.0,1048.0,179.0,11.0,0.0,9.0,2.0,0.0,GCA_905176675


In [11]:
phagesdf[['Total Predicted Prophages', 'Not enough genes', 'No phage genes', 'Kept']].agg('sum')

Total Predicted Prophages    21562059.0
Not enough genes             15047785.0
No phage genes                3248821.0
Kept                          3265453.0
dtype: float64

# What is in common between the groups?

In [9]:
gbkaa=set(gbk['assembly_accession'])
rastaa=set(rast['assembly_accession'])
gtdbaa=set(gtdb['assembly_accession'])
phagesaa=set(phagesdf['assembly_accession'])


gbr = gbkaa.intersection(rastaa)
gbg = gbkaa.intersection(gtdbaa)
gtr = gtdbaa.intersection(rastaa)

print(f"Between GBK and RAST there are {len(gbr):,} genomes in common")
print(f"Between GBK and GTDB there are {len(gbg):,} genomes in common")
print(f"Between GTDB and RAST there are {len(gtr):,} genomes in common")
print()

gbnotdone = gbkaa - phagesaa
gtnotdone = gtdbaa - phagesaa
ranotdone = rastaa - phagesaa
print(f"There are {len(gbnotdone):,} phages in Genbank that have not been analyzed")
print(f"There are {len(gtnotdone):,} phages in GTDB that have not been analyzed")
print(f"There are {len(ranotdone):,} phages in Genbank that have not been analyzed")
print()

allmissing = gbnotdone.intersection(gtnotdone).intersection(ranotdone)
print(f"There are {len(allmissing):,} phages in all three that have not been analyzed")

if False:
    with open("../data/unprocessed_phages.txt", 'w') as out:
        for o in allmissing:
            gbk[gbk['assembly_accession'] == 'GCA_002129805.1'][['assembly_accession', 'ftp_path']].to_csv(out, sep="\t", header=False, index=False)

Between GBK and RAST there are 244,870 genomes in common
Between GBK and GTDB there are 189,704 genomes in common
Between GTDB and RAST there are 140,463 genomes in common

There are 339,231 phages in Genbank that have not been analyzed
There are 65,392 phages in GTDB that have not been analyzed
There are 186,485 phages in Genbank that have not been analyzed

There are 52,219 phages in all three that have not been analyzed


In [10]:
print("IF WE IGNORE GENOME VERSIONS")
gbkaa=set(gbk['assembly_nover'])
rastaa=set(rast['assembly_nover'])
gtdbaa=set(gtdb['assembly_nover'])
phagesaa=set(phagesdf['assembly_nover'])


gbr = gbkaa.intersection(rastaa)
gbg = gbkaa.intersection(gtdbaa)
gtr = gtdbaa.intersection(rastaa)

print(f"There are {len(gbkaa):,} genomes in GenBank")
print(f"There are {len(rastaa):,} genomes in PATRIC")
print(f"There are {len(gtdbaa):,} genomes in GTDB")

print(f"Between GenBank and PATRIC there are {len(gbr):,} genomes in common")
print(f"Between GenBank and GTDB there are {len(gbg):,} genomes in common")
print(f"Between GTDB and PATRIC there are {len(gtr):,} genomes in common")
print()

gbnotdone = gbkaa - phagesaa
gtnotdone = gtdbaa - phagesaa
ranotdone = rastaa - phagesaa
print(f"There are {len(gbnotdone):,} phages in Genbank that have not been analyzed")
print(f"There are {len(gtnotdone):,} phages in GTDB that have not been analyzed")
print(f"There are {len(ranotdone):,} phages in PATRIC that have not been analyzed")
print()

allmissing = gbnotdone.intersection(gtnotdone).intersection(ranotdone)
print(f"There are {len(allmissing):,} phages in all three that have not been analyzed")

IF WE IGNORE GENOME VERSIONS
There are 904,681 genomes in GenBank
There are 320,033 genomes in PATRIC
There are 191,521 genomes in GTDB
Between GenBank and PATRIC there are 249,334 genomes in common
Between GenBank and GTDB there are 191,370 genomes in common
Between GTDB and PATRIC there are 143,445 genomes in common

There are 337,599 phages in Genbank that have not been analyzed
There are 64,358 phages in GTDB that have not been analyzed
There are 183,308 phages in PATRIC that have not been analyzed

There are 53,262 phages in all three that have not been analyzed
